In [ ]:
from collections import defaultdict, Counter
import itertools
import numpy as np
import json
import random as rd
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode, iplot

## Make a dataframe


In [ ]:
# Uncomment to upload a dataset of your own
# import google.colab.files;google.colab.files.upload();

In [ ]:
# read the .csv you uploaded with pandas. pandas has lots of options
# to adjust to different formats of datafile: sep (separator), skiprows, index_col and encoding (try latin1)
# are especially useful, as is `read_excel`

# here we read in file directly from our github
df = pd.read_csv("https://raw.githubusercontent.com/seasonsOfTheSun/convenient_data_storage/main/pbmc_processed.csv", index_col=0)

In [ ]:
# examine the dataframe
# check that features are on the columns
# and samples/observations are along the rows/indices
df

### Make a Network using UMAP

In [ ]:
!pip install umap-learn==0.5.3

In [ ]:
# creates the underlying network that is always generated (but usually hidden)
# whenever UMAP is called.
import umap.umap_
rndstate = np.random.RandomState(108)
umap_spmat,_,_ = umap.umap_.fuzzy_simplicial_set(df, 5, rndstate, "euclidean")
G = nx.from_scipy_sparse_array(umap_spmat)

## Generic Network Layout

The spring layout is the most generic possible layout, but can improved somwhat by initialising with a spectral layout.

In [ ]:
# Intialise a spring layout with a spectral layout
# to improve convergence speed and make better layout
init_pos = nx.spectral_layout(G, dim=3)
pos = nx.spring_layout(G, pos=init_pos, dim=3)

### Detect and Color Communities

In [ ]:
# Creating an empty dictionary to store node-community mapping
node_community = {}
# Iterating over the communities generated using Louvain algorithm
for i, v in enumerate(nx.community.louvain_communities(G)):
  # Iterating over the nodes in each community
  for j in v:
    # Assigning the community index 'i' to the node 'j'
    node_community[j] = i
# At this point, the 'node_community' dictionary contains
# the mapping of each node to its corresponding community's index.

In [ ]:
import numpy as np  # Importing the numpy library for generating random numbers

# Generate a color map dictionary where each specified key is mapped to a random hexadecimal color
# Format the RGB values into a hexadecimal color code and return it in uppercase
def generate_random_hex():
  rgb = np.random.randint(0, 255, size=(3,))
  return '#{:02x}{:02x}{:02x}'.format(*rgb).upper()

# Generate a color map dictionary where each specified key is mapped to a random hexadecimal color
def generate_cmap(key):
  return {i: generate_random_hex() for i in key}

# Generate a color map using the node_community values
cmap = generate_cmap(node_community)
# Assign colors to each node based on their corresponding community value using the color map
node_colors = {i: cmap[v] for i, v in node_community.items()}



## Plotly Preview, and Select Eigenvectors


### Define The Function `plotly_preview`

In [ ]:

import plotly.offline
def plotly_preview(G, pos, node_colors=None, edge_colors=None):

    """
    Generate a 3D network visualization using Plotly.

    Parameters:
        - G (networkx.Graph): The graph object representing the network.
        - pos3D (dict): A dictionary mapping each node to its 3D coordinates (x, y, z).
        - node_colors (dict, optional): A dictionary mapping nodes to custom hex colors.
                                        If not provided, the default color is '#40b9d4'.
        - edge_colors (dict, optional): A dictionary mapping edges to custom hex colors.
                                        If not provided, the default color is 'gray'.

    Output:
        - An HTML file named 'network_visualization.html' is generated, which opens in a web browser.

    Example usage:
        node_colors = {1: '#ff0000', 2: '#00ff00', 3: '#0000ff'}
        edge_colors = {(1, 2): '#ff00ff', (2, 3): '#ffff00'}
        pos = nx.spring_layout(G,dim=3)
        plotly_review_2(G, pos, node_colors, edge_colors)
    """

    # Create a Plotly figure
    fig = go.Figure()

    # Add nodes to the figure
    for node in G.nodes():
        x, y, z = pos[node]
        color = node_colors[node] if node_colors and node in node_colors else '#40b9d4'
        fig.add_trace(go.Scatter3d(
            x=[x],
            y=[y],
            z=[z],
            mode='markers',
            marker=dict(
                size=5,
                color=color,
            ),
            name=str(node),
            text=str(node),
            hovertemplate=None,
        ))

    # Add edges to the figure
    for edge in G.edges():
        x0, y0, z0 = pos[edge[0]]
        x1, y1, z1 = pos[edge[1]]
        edge_color = edge_colors[edge] if edge_colors and edge in edge_colors else 'gray'
        fig.add_trace(go.Scatter3d(
            x=[x0, x1],
            y=[y0, y1],
            z=[z0, z1],
            mode='lines',
            line=dict(
                color=edge_color,
                width=1,
            ),
            hoverinfo='none',
        ))

    # Set layout options
    fig.update_layout(
        scene=dict(
            xaxis=dict(visible=False),
            yaxis=dict(visible=False),
            zaxis=dict(visible=False),
        ),
        showlegend=False,
        hovermode='closest',
        margin=dict(l=0, r=0, b=0, t=0),
    )

    # Display the plot inline in the notebook
    iplot(fig)
    plotly.offline.plot(fig, filename='plotly_plot.html')


### Use the Function

In [ ]:
# Uncomment to see a preview of the full network = you may  is too big for preview to properly handle!
# plotly_preview(G,pos,node_colors=node_colors)

In [ ]:
plotly_preview(G,pos,node_colors=node_colors)

In [ ]:
import google.colab.files
google.colab.files.download("plotly_plot.html")


## Export JSON




### Define Function `make_json`

In [ ]:

def make_json(name, network, positions, node_color = '#40b9d4', link_color = '#999999', annotations = 'None', communities = 'None'):
    """
    Generates a JSON file from a given network graph using the specified parameters.

    Args:
        name (str, optional): Name of the graph.
        network (networkx.Graph): Network graph object.
        positions (dict): Dictionary mapping node IDs to their positions.
        node_color (dict): Dictionary mapping node IDs to their (hex-)colors.
        link_color (str or dict): (Hex-)color value for all links in the graph or dict with node tuple as key and hex color as value.
        communities (dict): 'None' for no communities (default) or dictionary mapping node IDs to their corresponding community ID.
        annotations (dict): Dictionary mapping node IDs to a list of annotations.

    Returns:
        None

    """

    # --------------------------
    # Generate VR GRAPH
    # --------------------------
    GVR = nx.Graph()
    GVR.graph['name'] = name

    # --------------------------------------
    # LOOKUP FOR NODE NAMES INTO IDs and vv
    # --------------------------------------
    d_idx_node = {}
    d_node_idx = {}
    for i, node in enumerate(sorted(network.nodes())):
        d_idx_node[i] = node
        d_node_idx[node] = i
    GVR.add_nodes_from(d_idx_node.keys())

    for edge in network.edges()(data=True):
        GVR.add_edge(d_node_idx[edge[0]],d_node_idx[edge[1]])

    # --------------------------
    # POS
    # --------------------------
    if isinstance(positions[next(iter(positions))], list):
        pass
    else:
        for key in positions:
            positions[key] = positions[key].tolist()

    posG = {d_node_idx[node]: list(xyz) for node, xyz in positions.items()}
    nx.set_node_attributes(GVR, posG, name="pos")

    # # --------------------------
    # # CLUSTER
    # # --------------------------
    if communities == 'None':
        dict_for_cluster = dict(zip(d_idx_node.keys(), [0 for _ in d_idx_node.keys()]))
    else:
        d_VRids_cluster = {d_node_idx[node]: str(cl_id) for node, cl_id in communities.items()}
        nx.set_node_attributes(GVR, d_VRids_cluster, name="cluster")


    # --------------------------
    # NODE COLOR
    # --------------------------
    d_node_colors={}

    if isinstance(node_color, dict):
        for nodeid in GVR.nodes():
            d_node_colors[nodeid] = node_color[d_idx_node[nodeid]]
    else:
        for nodeid in GVR.nodes():
            d_node_colors[nodeid] = node_color

    nx.set_node_attributes(GVR, d_node_colors, name="nodecolor")

    # --------------------------
    # LINK COLOR
    # --------------------------
    if isinstance(link_color, dict):
        # for different link colors
        d_edge_color = {}
        for a,b in GVR.edges():
            try:
                color = link_color[(d_idx_node[a],d_idx_node[b])]
            except KeyError:
                color = link_color[(d_idx_node[b],d_idx_node[a])]
            d_edge_color[(a,b)] = color
    else:
        # for unique link colors
        d_edge_color = {}
        for a,b in GVR.edges():
            d_edge_color[(a,b)] = link_color

    nx.set_edge_attributes(GVR, d_edge_color, name="linkcolor")

    # --------------------------
    # NODE ANNOTATION
    # --------------------------
    if isinstance(annotations, dict):

        l_annotations = [[str(d_idx_node[nodeid])] + [ annotation for annotation in annotations[d_idx_node[nodeid]]] for nodeid in sorted(GVR.nodes())]
        d_annotations = dict(zip(sorted(GVR.nodes()), l_annotations))
    else:
        d_annotations = {nodeid: [str(d_idx_node[nodeid])] for nodeid in GVR.nodes()}

    nx.set_node_attributes(GVR, d_annotations, name="annotation")

    # --------------------------
    # MAKE JSON for uploader
    # --------------------------

    G_json = json.dumps(nx.node_link_data(GVR))

    with open(GVR.name, "w") as outfile:
        outfile.write(G_json)


### Run the Function



In [ ]:
make_json("feature_network.json", G, pos,node_color=node_colors,communities=node_community)

In [ ]:
import google.colab.files
google.colab.files.download("feature_network.json")